    Data Science Cancer Treatment - Aplicação de Deep Learning usando Python e Keras

In [ ]:
#Executei no cmd para monitorar
#!pip install tensorflow

In [ ]:
#Executei no cmd para monitorar
#!pip install keras

In [1]:
import scipy

In [2]:
from scipy.sparse import issparse

In [3]:
# Importa os pacotes
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
from keras.layers import Dense

In [5]:
from keras.optimizers import SGD

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
import numpy as np

In [10]:
import pandas as pd

In [12]:
import matplotlib.pyplot as plt

In [30]:
#Carga dos datasets
ds_test_text = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/test_text_original", delimiter="\|\|", engine="python")

In [31]:
ds_test_text

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung cance...
2,2,Abstract Background Non-small cell lung cance...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...
...,...,...
134,134,The splicing factor SF3B1 is the most commonly...
135,135,Mutations in genes encoding proteins involved ...
136,136,Precursor mRNA splicing is catalyzed by the sp...
137,137,Malignant mesothelioma (MM) is an aggressive n...


In [32]:
ds_training_text = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/training_text_original", delimiter="\|\|", engine="python")

In [33]:
ds_training_text

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...
...,...,...
134,134,The splicing factor SF3B1 is the most commonly...
135,135,Mutations in genes encoding proteins involved ...
136,136,Precursor mRNA splicing is catalyzed by the sp...
137,137,Malignant mesothelioma (MM) is an aggressive n...


In [35]:
ds_test_variants = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/test_variants", delimiter=",", engine="python")

In [36]:
ds_test_variants

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A
...,...,...,...
5663,5663,SLC46A1,R113S
5664,5664,FOXC1,L130F
5665,5665,GSS,R267W
5666,5666,CTSK,G79E
